<a href="https://colab.research.google.com/github/srigit-dot/machine-learning/blob/main/AIPoweredChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from langchain.llms.base import LLM
from typing import Optional, List
import requests

# Define custom LLM for OpenRouter
class OpenRouterLLM(LLM):
    model: str
    api_key: str

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}]
        }

        response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=data)

        try:
            result = response.json()
            if "choices" not in result:
                raise ValueError(f"API Error: {result.get('error', result)}")
            return result["choices"][0]["message"]["content"]
        except Exception as e:
            print("🚨 Error from OpenRouter API:")
            print(response.text)
            raise e

    @property
    def _llm_type(self) -> str:
        return "openrouter"

# API Key
API_KEY = "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"

# Define models
deepseek_llm = OpenRouterLLM(api_key=API_KEY, model="deepseek/deepseek-chat")
gemini_llm = OpenRouterLLM(api_key=API_KEY, model="google/gemini-pro")

# LangChain setup
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question:\n{question}"
)

# Create chains
deepseek_chain = LLMChain(llm=deepseek_llm, prompt=prompt)
gemini_chain = LLMChain(llm=gemini_llm, prompt=prompt)

# Chat loop
print("💬 Chat with DeepSeek or Gemini! Type 'exit' to quit.")
model_choice = input("👉 Choose your model (deepseek / gemini): ").strip().lower()

if model_choice not in ["deepseek", "gemini"]:
    print("❌ Invalid model choice.")
else:
    chain = deepseek_chain if model_choice == "deepseek" else gemini_chain
    while True:
        question = input("\nYou: ")
        if question.lower() in ["exit", "quit"]:
            print("👋 Goodbye!")
            break
        try:
            response = chain.run(question)
            print(f"{model_choice.capitalize()}: {response}")
        except Exception as e:
            print(f"⚠️ Error: {e}")


💬 Chat with DeepSeek or Gemini! Type 'exit' to quit.
👉 Choose your model (deepseek / gemini): gemini

You: hi
Gemini: Hello! How can I help you today?

You: how are you
Gemini: I am doing well, thank you for asking. How are you doing today?

You: exit
👋 Goodbye!
